  
   ### Calculate Steady State Values

This function calculates the steady state values of capital per effective worker ($\tilde{k}^*$), human capital per effective worker ($\tilde{h}^*$), and output per effective worker ($\tilde{y}^*$) in a Solow model extended to include human capital.

#### Parameters:
- `sK` (float): Savings rate for physical capital.
- `sH` (float): Savings rate for human capital.
- `delta` (float): Depreciation rate.
- `alpha` (float): Output elasticity with respect to physical capital.
- `phi` (float): Output elasticity with respect to human capital.
- `n` (float): Population growth rate.
- `g` (float): Technological growth rate.

#### Returns:
- A dictionary containing steady state values for $\tilde{k}^*$, $\tilde{h}^*$, $\tilde{y}^*$.


In [1]:
def steady_state(sK, sH, delta, alpha, phi, n, g):
    # Solving for k_tilde_star and h_tilde_star from steady state equations
    A = (sK / (n + g + delta)) ** (1 / (1 - alpha - phi))
    B = (sH / (n + g + delta)) ** (1 / (1 - alpha - phi))
    k_tilde_star = A ** (phi / (alpha * (1 - phi - alpha)))
    h_tilde_star = B ** (alpha / (phi * (1 - phi - alpha)))
    
    # Calculate y_tilde_star using the Cobb-Douglas production function
    y_tilde_star = k_tilde_star ** alpha * h_tilde_star ** phi
    
    return {'k_tilde_star': k_tilde_star, 'h_tilde_star': h_tilde_star, 'y_tilde_star': y_tilde_star}

# Example parameters
params = {
    'sK': 0.3,  # Physical capital savings rate
    'sH': 0.2,  # Human capital savings rate
    'delta': 0.05,  # Depreciation rate
    'alpha': 0.3,  # Output elasticity of physical capital
    'phi': 0.2,   # Output elasticity of human capital
    'n': 0.01,    # Population growth rate
    'g': 0.02     # Technological growth rate
}

# Calculate steady state values
steady_states = steady_state(**params)
print(steady_states)


{'k_tilde_star': 33.94298617754171, 'h_tilde_star': 244.1406250000001, 'y_tilde_star': 8.644761670141202}
